In [13]:
import geopandas as gpd
import requests
import uuid
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
load_dotenv()

True

In [14]:
r = requests.get('https://skimap.org/skiareas/index.geojson')


In [15]:
gdf = gpd.read_file(r.text, driver='GeoJSON')

In [16]:
#Break out Nordic/Downhill to True False Columns
gdf['alpine'] = gdf['activities'].apply(lambda x: True if 'downhill' in x else False)
gdf['nordic'] = gdf['activities'].apply(lambda x: True if 'nordic' in x else False)
#Drop the original activities column
gdf.drop(columns=['activities'], inplace=True)
#gdf['nordic'].value_counts()


In [17]:
gdf = gdf.rename(columns={'status':'operatingstatus','official_website':'website'})

In [18]:
gdf['skiarea_pk'] = [uuid.uuid4() for _ in range(len(gdf.index))]
gdf['passaffiliation'] = 'Unaffiliated'
gdf['partnered'] = None

In [19]:
gdf = gdf[['skiarea_pk','id','scalerank','passaffiliation','partnered','name','operatingstatus','alpine','nordic','website','geometry']]

In [20]:
gdf.rename_geometry('location',inplace=True)

In [21]:
#Write the data to PostGIS
db_connection_url = os.getenv('DATABASE_URL')

# Create a SQLAlchemy engine
engine = create_engine(db_connection_url)
# Write the GeoDataFrame to PostGIS
gdf.to_postgis(name='ski_resorts_skiarea', con=engine, if_exists='append')



In [ ]:
gdf = gdf.where(gdf.notna(), None)
